# Datum a čas

Mnoho datových sad obsahuje nějaké časové údaje. Např. :term{cs="časové razítko" en="time stamp"} je údaj, které indikuje, kdy byla data uložena nebo kdy se nějaká událost stala. Tato informace je klíčová pro řadu analýz, jako je sledování trendů v čase, sezónní analýza, předpovídání a časové řady. Pandas umožňuje řadu operací, jak manipulovat s datem a časem. Při načtení ze souboru je ale nutné na začátku provést převod série na typ `datetime`, aby bylo jasné, že se jedná o časový údaj a nikoli o řetězec.

Uvažujme data o monitorování příjmu nějakého (např. televizního) signálu. Máme data o tom, kdy došlo ke ztrátě signálu (začátek výpadku) a kdy byl signál obnoven (konec výpadku). data jsou uložena v souboru [signal_monitoring.csv](https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/datum-cas-shift/datum-cas/signal_monitoring.csv)

In [1]:
import pandas as pd

signal_monitoring = pd.read_csv("https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/datum-cas-shift/datum-cas/signal_monitoring.csv")
signal_monitoring["event_date_time"] = pd.to_datetime(signal_monitoring["event_date_time"])

Type `datetime` má sadu vlastností, pomocí kterých můžeme získat určitou část z data a času. Například vlastnost `dt.date` obsahuje pouze datum, tj. "ořeže" údaj o čas. Pomocí této vlastnosti pak můžeme například vypočítat počet výpadků za jednotlivé dny. Budeme pracovat pouze se řádky, které evidují ztrátu signálu, tj. sloupec `event_type` obsahuje hodnotu `signal lost`. Nakonec můžeme použít metodu `.values_count()`.

In [2]:
signal_monitoring_signal_lost = signal_monitoring[signal_monitoring["event_type"] == "signal lost"]
signal_monitoring_signal_lost = signal_monitoring_signal_lost.reset_index()
signal_monitoring_signal_lost["date"] = signal_monitoring_signal_lost["event_date_time"].dt.date
signal_monitoring_signal_lost["date"].value_counts()

date
2021-02-09    2
2021-02-12    2
2021-02-03    1
2021-02-05    1
2021-02-06    1
2021-02-08    1
2021-02-13    1
2021-02-15    1
2021-02-16    1
2021-02-18    1
2021-02-19    1
2021-02-22    1
Name: count, dtype: int64

Dále můžeme například pomocí vlastnosti `dt.dayofweek` den v týdnu (jako číslo, kde 0 označuje pondělí a 6 neděli), `dt.month` vrátí měsíc jako číslo (stejné jako v kalendáři, tj. leden je 1), `dt.hour` naopak vrací hodinu atd. Můžeme tedy sledoval, jak často se výpadky vyskytují v rámci denní doby, měsíce, dne v týdnu atd.

## Metoda shift

Uvažujme nyní, že chceme vypočítat délku nějakého výpadku. Určitou komplikací je, že začátek a konec jednoho výpadku je na různých řádcích, nemůžeme tedy postupovat jako při běžném výpočtu. Můžeme ale použít metodu `shift()`, která umí data v jednom sloupci posunout nahoru nebo dolů.

In [30]:
signal_monitoring = pd.read_csv("https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/datum-cas-shift/datum-cas/signal_monitoring.csv")
signal_monitoring["event_date_time"] = pd.to_datetime(signal_monitoring["event_date_time"])
signal_monitoring["date"] = signal_monitoring["event_date_time"].dt.date

Nyní použijeme metodu `shift()` na sloupec `event_date_time`. Pomocí metody pak přidáme k tabulce nový sloupec. Nejdůležitějším parametrem metody je parametr periods, který může mít kladnou nebo zápornou hodnotu.

- Kladná hodnota parametru periods znamená, že hodnoty budou posunuty směrem dolů.
- Záporná hodnota parametru periods znamená, že hodnoty budou posunuty směrem nahoru.

Pro náš případ bude ideální, pokud posuneme hodnoty sloupce `event_date_time` o jeden řádek směrem nahoru. Tím zajistíme, že pokud má sloupec `event_type` hodnotu _signal lost_, uvidíme v jednom řádku začátek i konec výpadku. Tím pádem bude stačit tyto hodnoty od sebe odečíst. Pro `event_type` _signal restored_ nebude mít tato hodnota smysl, ale to nevadí, tyto řádky můžeme pomocí dotazu z tabulky odfiltrovat.


In [31]:
signal_monitoring["event_end_date_time"] = signal_monitoring["event_date_time"].shift(periods=-1)

Opět v datech ponecháme pouze řádky, které mají ve sloupci `event_type` hodnotu _signal lost_.

In [32]:
signal_monitoring_signal_lost = signal_monitoring[signal_monitoring["event_type"] == "signal lost"]

Nyné můžeme spočítat rozdíl mezi začátkem výpadku a koncem výpadku, který udává jeho délku.

In [35]:
signal_monitoring_signal_lost["outage_length"] = signal_monitoring["event_end_date_time"] - signal_monitoring["event_date_time"]

C:\Users\Jakub\AppData\Local\Temp\ipykernel_13164\3944047271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signal_monitoring_signal_lost["outage_length"] = signal_monitoring["event_end_date_time"] - signal_monitoring["event_date_time"]


Typ hodnoty ve sloupci `outage_length` je označovaný jako `timedelta`. Tento typ hodnoty označuje rozdíl mezi dvěma hodnotami typu `datetime`.

In [36]:
signal_monitoring_signal_lost.groupby("date")["outage_length"].sum()

date
2021-02-03   0 days 00:34:03
2021-02-05   0 days 00:32:10
2021-02-06   0 days 00:16:50
2021-02-08   0 days 00:39:09
2021-02-09   0 days 00:44:38
2021-02-12   0 days 01:02:07
2021-02-13   0 days 00:15:34
2021-02-15   0 days 00:50:33
2021-02-16   0 days 00:13:26
2021-02-18   0 days 00:47:35
2021-02-19   0 days 00:12:33
2021-02-22   0 days 00:59:46
Name: outage_length, dtype: timedelta64[ns]

Příklad řešení s využitím ChatGPT je [zde](https://chat.openai.com/share/eb92296b-1968-4387-9cc4-592023d4d104).

---
## Cvičení

### Půlmaraton

Uvažuj časy závodníků za ročníky půlmaratonu 2019 a 2020, které jsou uloženy v souboru [half_marathon.csv](https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/datum-cas-shift/excs/pulmaraton/half_marathon.csv). V souboru je uloženo jméno závodníka (závodnice), rok narození, jeho/její čas a rok závodu, ke kterému se čas vztahuje. Tvým úkolem je spočítat, o kolik se změnil průměrný čas každého ze závodníků a závodnic a zda se v průměru zlepšili či zhoršili (například protože kvůli lockdownům méně trénovali).

Můžeš využít následující postup:

- Převeď sloupec s časem závodníka na typ datetime. Použij stejný postup, jaký jsme si ukázali v minulé v lekci. Protože jde pouze o časový údaj, pandas k němu připojí dnešní datum, aby byly ve sloupci datum i čas. Toho si ale nevšímej, u obou sloupců je datum stejný, takže na porovnání údajů to nebude mít vliv.
- Pomocí metody `shift()` si dej na jeden řádek výsledky obou závodů. Metodu `shift()` použij tak, aby nový sloupec obsahoval hodnoty posunuté o jeden řádek dolů. Je třeba nahradit X vhodně zvoleným číslem. Poté si nech v datech pouze data, která mají ve sloupci `Rok zavodu` hodnotu 2020. Řádky, které mají ve sloupci `Rok zavodu` hodnotu 2019, totiž obsahují "pomíchané" hodnoty dvou různých závodníků.
- Vypočítej rozdíl mezi časy závodníka a převeď ho na sekundy (postup jsme si ukazovali v lekci). Dále spočítej průměrnou změnu. Vyšlo i kladné nebo záporné číslo? A co to znamená?

In [37]:
df = pd.read_csv("https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/datum-cas-shift/excs/pulmaraton/half_marathon.csv")
df.head()

,Jmeno,Rocnik,Cas,Rok zavodu
0,Aster Vladimír,1965,01:53:25,2019
1,Aster Vladimír,1965,02:00:19,2020
2,Asterová Jana,1974,02:16:59,2019
3,Asterová Jana,1974,02:30:01,2020
4,Baborová Anna,1990,01:58:20,2019


In [38]:
df["Cas"] = pd.to_datetime(df["Cas"])
df["Cas 2019"] = df["Cas"].shift(1)
df = df[df["Rok zavodu"] == 2020]
df["Zmena"] = df["Cas"] - df["Cas 2019"]
df

C:\Users\Jakub\AppData\Local\Temp\ipykernel_13164\1170363246.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Cas"] = pd.to_datetime(df["Cas"])


,Jmeno,Rocnik,Cas,Rok zavodu,Cas 2019,Zmena
1,Aster Vladimír,1965,2024-04-07 02:00:19,2020,2024-04-07 01:53:25,0 days 00:06:54
3,Asterová Jana,1974,2024-04-07 02:30:01,2020,2024-04-07 02:16:59,0 days 00:13:02
5,Baborová Anna,1990,2024-04-07 01:52:25,2020,2024-04-07 01:58:20,-1 days +23:54:05
7,Bambas Jan,1975,2024-04-07 02:35:39,2020,2024-04-07 02:02:59,0 days 00:32:40
9,Barochovská Andrea,1976,2024-04-07 02:50:05,2020,2024-04-07 03:01:25,-1 days +23:48:40
...,...,...,...,...,...,...
180,Šulcová Jitka,1974,2024-04-07 01:59:07,2020,2024-04-07 01:43:12,0 days 00:15:55
182,Švarcová Petra,1974,2024-04-07 02:50:01,2020,2024-04-07 02:27:45,0 days 00:22:16
184,Švelch Rosťa,1966,2024-04-07 01:55:11,2020,2024-04-07 01:54:58,0 days 00:00:13
186,Žitková Jana,1974,2024-04-07 02:42:48,2020,2024-04-07 02:45:54,-1 days +23:56:54


In [39]:
df["Zmena"].mean()

Timedelta('0 days 00:09:36.031578947')

### Swing states

V případě amerických prezidentských voleb obecně platí, že ve většině států dlouhodobě vyhrávají kandidáti jedné strany. Například v Kalifornii vyhrávají poměrně dlouho kandidáti Demokratické strany, v Texasu zase kandidáti Republikánské strany. Státy, kde se vítězné strany střídají, jsou označovány jako _swing states_ ("kolísavé státy"). Tvým úkolem je pro jeden konkrétní stát spočítat, kolikrát v něm došlo ke změně vítězné strany, přičemž data jsou uložena v souboru [election-data.csv](https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/datum-cas-shift/excs/swing-states/election-data.csv).

V souboru jsou důležité následující sloupce:

`year` - rok voleb,
`state` - stát,
`party_simplified` - zjednodušené označení politické strany,
`rank` - pořadí kandidáta v rámci státu a roku.

Vyber si jeden stát, např. stát PENNSYLVANIA. Vytvoř si tabulku, kde budou data pouze za tento stát. Současně si vyber pouze data o vítězích, tj. řádky, které mají ve sloupci `rank` hodnotu `1`. Vytvoř sloupec, který bude obsahovat politickou strunu vítěze voleb z přechozího volebního období. Pokud spočítáš řádky, kde se politická strana současného vítěze liší od strany minulého, zjistíš, kolikrát voliči v daném státě přešli od jedné strany ke druhé. Pozor na to, že pro rok 1976 neznáme předchozího vítěze, v nově přidaném sloupci tedy bude prázdná hodnota. Té se můžeš zbavit například pomocí metody `dropna()`.

Např. pro stát PENNSYLVANIA by ti mělo vyjít, že ke změně došlo celkem čtyřikrát, a to v letech 1980, 1992, 2016 a 2020. Ve státě SOUTH CAROLINA došlo ke změně pouze jednou, a to v roce 1980, kdy vyhrál Ronald Reagan. Od té doby zde vyhrávají pouze kandidáti Republikánské strany.

In [40]:
data = pd.read_csv("https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/datum-cas-shift/excs/swing-states/election-data.csv")

data = data[(data["state"] == "SOUTH CAROLINA") & (data["rank"] == 1)]
data["previous_winner_party"] = data["party_simplified"].shift(1)
data = data.dropna()
data[data["party_simplified"] != data["previous_winner_party"]]

,year,state,candidate,party_simplified,candidatevotes,rank,previous_winner_party
3396,1980,SOUTH CAROLINA,"REAGAN, RONALD",REPUBLICAN,439277,1.0,DEMOCRAT
